### Create a GIF of Landsat images 

inspired from https://stackstac.readthedocs.io/en/latest/examples/gif.html


In [1]:
from dask.distributed import LocalCluster,Client
import dask.utils
import pystac_client
import stackstac
import dask.array as da
from geogif import gif, dgif
import geopandas as gpd
import planetary_computer as pc
import hvplot


from IPython.display import HTML, display
import folium
import folium.plugins
from branca.element import Figure
import shapely.geometry

In [4]:
# port forwarding : ssh -L 8000:localhost:8787 protect
# ou
# ssh -L 9998:129.88.193.194:22 ssh-ige
# ssh -L 9999:localhost:8787 -N -p 9998 localhost
# opne localhost:9999

cluster = LocalCluster(n_workers=20,
                       threads_per_worker=1,
                       dashboard_address=8787,
                       memory_limit='2GB')
   
#cluster = LocalCluster()   
client = Client(cluster)

display(client)

/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33521 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:33521/status,
Dashboard: http://127.0.0.1:33521/status,Workers: 20
Total threads: 20,Total memory: 37.25 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46461,Workers: 20
Dashboard: http://127.0.0.1:33521/status,Total threads: 20
Started: Just now,Total memory: 37.25 GiB
Comm: tcp://127.0.0.1:39247,Total threads: 1
Dashboard: http://127.0.0.1:34521/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:33489,


In [ ]:
from dask.utils import ensure_dict, format_bytes
    
wk = client.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)

In [7]:
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

In [8]:
#petermann - or create one with https://geojson.io/#map=2/20.0/0.0 and follow https://aws.amazon.com/fr/blogs/apn/transforming-geospatial-data-to-cloud-native-frameworks-with-element-84-on-aws/

#filename = "geojson file path"# read in AOI as a GeoDataFrame
filename = "petermann.geojson"# read in AOI as a GeoDataFrame
# read in AOI as a GeoDataFrame
aoi = gpd.read_file(filename)

fig = Figure(width="400px", height="500px")
map1 = folium.Map()
fig.add_child(map1)

folium.GeoJson(
    aoi['geometry'],
    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    name="Query",
).add_to(map1)


map1.fit_bounds(bounds=convert_bounds(aoi.unary_union.bounds))
display(fig)

With the pystac_client module’s Client class, Open the STAC API. 

In [9]:
# see : https://github.com/microsoft/PlanetaryComputerExamples/blob/main/datasets/landsat-8-c2-l2/landsat-8-c2-l2-example.ipynb

bbox =aoi.unary_union.bounds

LandsatSTAC = pystac_client.Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')

#for collection in LandsatSTAC.get_collections():
    #print(collection)

search = (
    LandsatSTAC
    .search(
        bbox=bbox,
        query =  {"eo:cloud_cover":{"lt":15}},
        datetime = "2018-01-01/2018-06-30", 
        collections = ["landsat-8-c2-l2"]
               
    )
)

In [10]:
items = pc.sign(search)
print(str(len(items))+ ' scenes found')

244 scenes found


Turn STAC items into xarray as a temporal stack, using stackstac.


In [14]:
stack = stackstac.stack(items, chunksize=(6, 19, 405, 424),bounds_latlon=bbox, epsg = 32620, resolution=200)
stack

,Array,Chunk
Bytes,5.93 GiB,149.35 MiB
Shape,"(244, 19, 405, 424)","(6, 19, 405, 424)"
Count,3 Graph Layers,41 Chunks
Type,float64,numpy.ndarray


In [15]:
# use common_name for bands
stack = stack.assign_coords(band=stack.common_name.fillna(stack.band).rename("band"))
stack.band

<xarray.DataArray 'band' (band: 19)>
array(['coastal', 'blue', 'green', 'red', 'nir08', 'swir16', 'swir22', 'ST_QA',
       'lwir11', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_TRAD', 'ST_URAD',
       'QA_PIXEL', 'ST_ATRAN', 'ST_CDIST', 'QA_RADSAT', 'SR_QA_AEROSOL'],
      dtype=object)
Coordinates: (12/14)
  * band                       (band) object 'coastal' ... 'SR_QA_AEROSOL'
    description                (band) <U91 'Collection 2 Level-2 Coastal/Aero...
    instruments                object {'tirs', 'oli'}
    view:off_nadir             int64 0
    landsat:processing_level   <U4 'L2SP'
    landsat:collection_number  <U2 '02'
    ...                         ...
    gsd                        (band) float64 30.0 30.0 30.0 ... 30.0 30.0 30.0
    title                      (band) <U46 'Coastal/Aerosol Band (B1)' ... 'A...
    common_name                (band) object 'coastal' 'blue' ... None None
    center_wavelength          (band) object 0.44 0.48 0.56 ... None None None
    full_width_half_max        (band) object 0.02 0.06 0.06 ... None None None
    epsg                       int64 32620

In [16]:
#See how much input data there is for just RGB. This is the amount of data we’ll end up processing
stack.sel(band=["red", "green", "blue"])

,Array,Chunk
Bytes,0.94 GiB,23.58 MiB
Shape,"(244, 3, 405, 424)","(6, 3, 405, 424)"
Count,4 Graph Layers,41 Chunks
Type,float64,numpy.ndarray


In [17]:
# Make a bitmask---when we bitwise-and it with the data, it leaves just the 4 bits we care about
mask_bitfields = [1, 2, 3, 4]  # dilated cloud, cirrus, cloud, cloud shadow
bitmask = 0
for field in mask_bitfields:
    bitmask |= 1 << field

bin(bitmask)

'0b11110'

In [18]:
qa = stack.sel(band="QA_PIXEL").astype("uint16")
bad = qa & bitmask  # just look at those 4 bits

good = stack.where(bad == 0)  # mask pixels where any one of those bits are set

In [ ]:
# What's the typical interval between scenes?
#good.time.diff("time").dt.days.plot.hist();

In [19]:
# keep rgb bands + Make annual median composites (`Q` means 2 quarters)
composites = good.sel(band=["red", "green", "blue"]).resample(time="M").median("time")
composites#.chunk(chunks=(2,3,405,424))


<xarray.DataArray 'stackstac-ec2680481db7683090b25ce2110e16e5' (time: 3,
                                                                band: 3,
                                                                y: 405, x: 424)>
dask.array<stack, shape=(3, 3, 405, 424), dtype=float64, chunksize=(1, 2, 279, 292), chunktype=numpy.ndarray>
Coordinates:
  * band                       (band) object 'red' 'green' 'blue'
  * x                          (x) float64 4.966e+05 4.968e+05 ... 5.812e+05
  * y                          (y) float64 9.013e+06 9.013e+06 ... 8.932e+06
    instruments                object {'tirs', 'oli'}
    view:off_nadir             int64 0
    landsat:processing_level   <U4 'L2SP'
    landsat:collection_number  <U2 '02'
    landsat:wrs_type           <U1 '2'
    platform                   <U9 'landsat-8'
    epsg                       int64 32620
  * time                       (time) datetime64[ns] 2018-04-30 ... 2018-06-30
Attributes:
    spec:        RasterSpec(epsg=32620, bounds=(496600, 8932000, 581400, 9013...
    crs:         epsg:32620
    transform:   | 200.00, 0.00, 496600.00|\n| 0.00,-200.00, 9013000.00|\n| 0...
    resolution:  200

In [20]:
%%time
singleimage= composites.sel(time='2018-04-30T00:00:00.000000000').compute()

2022-10-24 08:50:45,084 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.30 GiB -- Worker memory limit: 1.86 GiB
2022-10-24 08:50:45,177 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.30 GiB -- Worker memory limit: 1.86 GiB
2022-10-24 08:50:45,276 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

KilledWorker: Attempted to run task ('asset_table_to_reader_and_window-fetch_raster_window-a7ab37b75dbd460c8ef181683784b38e', 0, 0, 0, 0) on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:42235. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

2022-10-24 08:56:30,812 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.32 GiB -- Worker memory limit: 1.86 GiB
2022-10-24 08:56:30,913 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.32 GiB -- Worker memory limit: 1.86 GiB
2022-10-24 08:56:31,013 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [ ]:
singleimage.plot.imshow(rgb="band", robust=True)

In [ ]:
cleaned = composites.ffill("time")
cleaned.chunk((1,3,405,424))

In [ ]:
ts = cleaned.persist()
ts_local = ts.compute()


In [ ]:
ts_local.plot.imshow(col="time", rgb="band", col_wrap=5, robust=True)

In [ ]:
%time
#gif_img = dgif(rgb).compute()

In [ ]:
gif_data = dgif(ts,fps=8, bytes=True).compute()
with open("petermann_planetary_2016_2021.gif", "wb") as f:
        f.write(gif_data)
